# Web Scraping with BeautifulSoup

Sometimes there may not be an easily accessible data set for your project. However, there may be data that exists on the web which you can scrape. One way to do this in python is to use `BeautifulSoup`.

## What we will accomplish in this notebook

In this notebook we will:
- Discuss the structure of HTML code,
- Introduce the `bs4` pacakge,
- Parse simple HTML code with `BeautifulSoup`,
- Review how to request the HTML code from a url,
- Scrape data from an actual webpage and
- Touch on some of the issues that may arise when web scraping.

In [1]:
## Import base packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from seaborn import set_style
set_style("whitegrid")

## Scraping data with `BeautifulSoup`

### Importing `BeautifulSoup`

In order to use `BeautifulSoup` we first need to make sure that we have it installed on our computer. Try to run the following code chunks.

In [2]:
## this imports BeautifulSoup from its package, bs4
import bs4

In [3]:
## Run this to check your version
## I wrote this notebook with version  4.12.2
print(bs4.__version__)

4.12.2


If the above code does not work you will need to install the package before being able to run the code in this notebook. Here are installation instructions from the `bs4` documentation:
- Via conda: <a href="https://anaconda.org/conda-forge/bs4">https://anaconda.org/conda-forge/bs4</a>,
- Via pip: <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-beautiful-soup">https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-beautiful-soup</a>.

### The structure of an HTML page

`BeautifulSoup` takes in an HTML document and will 'parse' it for you so that you can extract the information you want. To best understand what that means we will look at a toy example of a webpage. To see what the snippet of HTML code below looks like in a web browser click here <a href="SampleHTML.html">SampleHTML.html</a>.

In [4]:
## This is an html chunk
## It has a head and a body, just like you
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
</body>
</html>
"""

We can now use `BeautifulSoup` to parse this simple HTML chunk.

In [5]:
## First we import the BeautifulSoup object
from bs4 import BeautifulSoup

In [6]:
## Now we make a BeautifulSoup object out of the html code
## The first input is the html code
## The second input is how you want BeautifulSoup
## to parse the code
soup = BeautifulSoup(html_doc, 'html.parser')

In [7]:
## Let's use the prettify method to make our html pretty and see what it has to say
## Ideally this is how someone writing pure html code would write their code
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



Html files have a natural tree structure that we will briefly cover now. Here is the tree of our sample HTML:

<img src = "html_tree.png" width = "50%"></img>

Each level in the tree represents a 'generation' of the html code. for instance the body has 3 p children, the leftmost p has one b child. `BeautifulSoup` helps us traverse these trees to gather the data we want.

In [ ]:
## Below are some examples of beautifulsoup methods and 
## attributes that help us better understand the structure 
## of html code

In [11]:
## We can traverse to the "title" by working our way through
## the tree
soup.head
soup.head.title

<title>The Dormouse's story</title>

In [17]:
## Notice we can also get the title like so
## This is because this is the first and only title 
## in the code
soup.title

<title>The Dormouse's story</title>

In [18]:
## What if I just want the text from the title?
soup.title.text

"The Dormouse's story"

In [19]:
## What html structure is the title's parent?
soup.title.parent

<head><title>The Dormouse's story</title></head>

In [20]:
## What is the first a of the html document?
soup.a

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [21]:
## What is the first a's class?
soup.a['class']

['sister']

In [23]:
## There are multiple a's can I find all of them?
soup.find_all('a')


for a in soup.find_all('a'):
    print()
    print(a['class'], a.text)


['sister'] Elsie

['sister'] Lacie

['sister'] Tillie


#### Exercises

Take a moment and try to complete the following exercises.

In [26]:
## Find the first p of the document
## What is the first p's class? What string is in that p?
soup.p

soup.p['class']

soup.p.text

"The Dormouse's story"

In [27]:
## For all of the a's in the document find their href

for a in soup.find_all('a'):
    print(a['href'])


http://example.com/elsie
http://example.com/lacie
http://example.com/tillie


## Scraping real webpages

Let's now pivot to a real webpage. In this example we will imagine we are in the spot of wanting to scrape information regarding the website FiveThirtyEight's Sports articles as found here, <a href="https://fivethirtyeight.com/sports/">https://fivethirtyeight.com/sports/</a>.

### Sending a request

In order to scrape that data we need to have the HTML code associated with the page. In python we can do this with the `requests` module.

In [28]:
import requests

In [29]:
## This is the url for the HTML code we want
url = "https://fivethirtyeight.com/sports/"

## We send a request to the website's server with
## the following code
requests.get(url)

<Response [200]>

Now we will want to store this response in a variable so we can access its contents. First we will note that, if the request was successful, we should be seeing `<Response [200]>` above. This tells us that the request was recieved and the data was returned successfully. If we instead saw something like `404` or `500`, we would know that something went wrong. For a list of possible response codes see <a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#server_error_responses">https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#server_error_responses</a>.

In [30]:
r = requests.get(url)

In [31]:
r.status_code

200

In [32]:
## The HTML code is stored in r.content
r.content

b'<!DOCTYPE html>\n<html lang="en-US" class="no-js">\n<head>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t\n\t\t<script>\n\t\t\ttargetPageParams = function() {\n\t\t\t\treturn {"page":{"page_id":359271,"page_name":"fivethirtyeight:sports:archive","section_1":"sports","section_2":"sports:sports:archive","content_type":"sports:archive","app_version":"1.0.28","device_type":"Desktop","prev_page":"https:\\/\\/fivethirtyeight.com\\/","template":"none"},"site":{"edition":"en-us","language":"en","site":"fivethirtyeight"},"device":{"device_type":"Desktop"},"at_property":"ced193d1-6804-5ebf-3643-6343d14fe8ca"};\n\t\t\t};\n\t\t</script>\n\t\t<script src="https://fivethirtyeight.com/wp-content/plugins/dtci-adobe-target/assets/js/at.js"></script> \t\t\t\t<link rel=\'dns-prefetch\' href=\'//dcf.espn.com\' />\n\n\t\t\n\t\t<script type="text/javascript">\n\t\t\tfunction setCountryCookie(value,days) {\n\t\t\t\tvar expires = "";\n\t\t\t\tif (days

In [33]:
## We can now parse this data with BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')

In [34]:
soup.head.title

<title>Sports – FiveThirtyEight</title>

### Web developer tools

As we can see, this is much messier than our simple example above. 

We only want three pieces of information for each article/podcast/posting listed on the page:
1. The title,
2. The author and
3. The associated url.

To hone in on this information we can utilize the web developer tools for your browser. Below are links on how to locate the web developer tools for Mozilla Firefox, Google Chrome and Safari:
- Mozilla Firefox: Go to `Browser Tools` in the `Tools` dropdown menu and click on `Web Developer Tools` you should see something like this:
<img src="firefox_develop.jpg" width="80%"></img>
- Google Chrome: Go to `Developer` in the `View` dropdown menu and click on `Developer Tools` you should see something like this:
<img src="chrome_develop.jpg" width="80%"></img>
- Safari: Go to the `Develop` dropdown menu and select `Show Web Inspector`, if there is no `Develop` dropdown menu follow the instructions on this page <a href="https://support.apple.com/guide/safari/use-the-developer-tools-in-the-develop-menu-sfri20948/mac">https://support.apple.com/guide/safari/use-the-developer-tools-in-the-develop-menu-sfri20948/mac</a>. You should see something like this once opened:
<img src="safari_develop.jpg" width="80%"></img>

<i>Note that the images above will be slightly different than what you see because 538 will have different articles at the time you access the page. These images were added 3-10-2022.</i>

<br>

The web developer tools will allow you to find out in what HTML elements certain pieces of data live. For example, you should be able to hover over an item on the webpage and it will highlight what HTML structure holds it. Here we can see what that looks like for the banner article (the large one at the top) of this page:
<img src="div_highlight.png" width="30%"></img>

We can use this information to get the data we desire.

In [36]:
## Getting the articles
soup.find('h2', {'class', "article-title"}).text

'\nBusting 4 Major Myths About Referees In The NBA Playoffs\n'

In [37]:
soup.find_all('h2', {'class', "article-title"})

[<h2 class="article-title entry-title">
 <a href="https://fivethirtyeight.com/features/busting-4-major-myths-about-referees-in-the-nba-playoffs/" name="&amp;lpos=fivethirtyeight&amp;lid=sports+mem:story1">Busting 4 Major Myths About Referees In The NBA Playoffs</a>
 </h2>,
 <h2 class="article-title entry-title">
 <a data-content-type="feature" data-post-id="359237" data-section-1="sports" data-section-2="sports:nba playoffs" href="https://fivethirtyeight.com/features/how-mikal-bridges-went-from-supporting-player-to-star/" name="&amp;lpos=fivethirtyeight&amp;lid=sports+features:story1">
 				How Mikal Bridges Went From Supporting Player To Star			</a>
 </h2>,
 <h2 class="article-title entry-title">
 <a data-content-type="feature" data-post-id="359231" data-section-1="sports" data-section-2="sports:mlb" href="https://fivethirtyeight.com/features/these-3-former-mlb-prospects-have-gone-from-busts-to-busting-out/" name="&amp;lpos=fivethirtyeight&amp;lid=sports+features:story2">
 				These 3

In [39]:
[h.text.strip() for h in soup.find_all('h2', {'class', "article-title"})]

['Busting 4 Major Myths About Referees In The NBA Playoffs',
 'How Mikal Bridges Went From Supporting Player To Star',
 'These 3 Former MLB Prospects Have Gone From Busts To Busting Out',
 'The Rays’ Dominance Goes Even Deeper Than Their 13-0 Start',
 'Who Won The 2023 WNBA Draft?',
 'Can Anyone Stop The Big, Bad Bruins?',
 'The Mets Assembled The Most Expensive Baseball Team Ever. Is That Enough To Make Them MLB’s Best Team?',
 'Our New Live In-Game Win Probabilities Are Here For The NBA Playoffs',
 'No Play-In Team Has Won An NBA Playoff Series. Will That Change This Season?',
 'It’s Rough Out There In Dallas. But It’s Not Kyrie Irving’s Fault.']

##### Exercise

Take some time to write code to retrieve all of the article authors below.

In [43]:
## code here
soup.find_all('p', {'class':"single-metadata card vcard"})

[<p class="single-metadata card vcard">By <a class="author url fn" href="https://fivethirtyeight.com/contributors/jared-dubin/" rel="author" title="">Jared Dubin</a></p>,
 <p class="single-metadata card vcard">By <a class="author url fn" href="https://fivethirtyeight.com/contributors/alex-kirshner/" rel="author" title="">Alex Kirshner</a></p>,
 <p class="single-metadata card vcard">By <a class="author url fn" href="https://fivethirtyeight.com/contributors/neil-paine/" rel="author" title="">Neil Paine</a></p>,
 <p class="single-metadata card vcard">By <a class="author url fn" href="https://fivethirtyeight.com/contributors/howard-megdal/" rel="author" title="">Howard Megdal</a></p>,
 <p class="single-metadata card vcard">By <a class="author url fn" href="https://fivethirtyeight.com/contributors/neil-paine/" rel="author" title="">Neil Paine</a> and <a class="author url fn" href="https://fivethirtyeight.com/contributors/terrence-doyle/" rel="author" title="">Terrence Doyle</a></p>,
 <p cla

In [42]:
## code here
[p.text for p in soup.find_all('p', {'class':"single-metadata card vcard"})]

['By Jared Dubin',
 'By Alex Kirshner',
 'By Neil Paine',
 'By Howard Megdal',
 'By Neil Paine and Terrence Doyle',
 'By Alex Kirshner',
 'By Ryan Best and Jay Boice',
 'By Jared Dubin',
 'By Louis Zatzman']

## Common problems while web scraping

### Messy or inconsistent HTML code

We have seen one problem that you can encounter while web scraping, small and messy differences in HTML code that make automating your scraping more difficult. It is important to note that 538 is actually not very messy in the grand scheme of the world wide web. For example, you can come across websites that do not label their HTML elements with `id`s or `class`es or any other kind of distinguishing meta data. This makes automation incredibly difficult. Other websites may offer no consistency from page to page. In such cases there may not be a quick or easy fix, you typically just have to hack something together and hope it works.

### Too many requests

Repeatedly sending requests to the same website can raise a flag at the site's server after which your IP address will be blocked from receiving future request results for some period of time. This is why it is good practice to space out your requests to a single website. You can do so with the `sleep` function in the `time` module, <a href="https://docs.python.org/3/library/time.html#time.sleep">https://docs.python.org/3/library/time.html#time.sleep</a>. While this decreases your risk of being flagged as a bot/scraper, it is also just being a good denizen of the internet. Sending too many requests to a single website in a short amount of time can mess with that website's ability to function for other visitors.

### Bot detection

Some websites have been set up to detect bot/scraper activity regardless of the number of times you send a request. Sometimes there are ways around this, but the specific approach depends upon how the website is blocking your request. To counter such detection do a web search for the specific error or response code you are getting and look for a helpful stack overflow or stackexchange post.

### User interactive content

Some of the content on a page may be dependent on the actions of a user visiting that page. For example, there are websites where data tables do not load until the user has clicked a button or scrolled down the page.

#### `selenium`

One way to access information that requires user input is with `selenium`, <a href="https://www.selenium.dev/">https://www.selenium.dev/</a>. `selenium` installation instructions can be found here, <a href="https://pypi.org/project/selenium/">https://pypi.org/project/selenium/</a>, and documentation on how to use the package can be found here, <a href="https://selenium-python.readthedocs.io/index.html">https://selenium-python.readthedocs.io/index.html</a>.

## Summary

In this notebook we touched on how you can parse HTML code with the `bs4` package. We looked at both a simple phony example and an example from a live website. If you are interested in learning more about `bs4` I encourage you to consult their documentation, <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">https://www.crummy.com/software/BeautifulSoup/bs4/doc/</a>.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)